# Importing libraries

In [2]:

import pandas as pd
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)



# Code 
** This Python script compares the order and courier charges data between Company X and the associated courier company. 
It involves data preprocessing, merging, calculations, and comparison to identify overcharged, correctly charged, and 
undercharged orders. The results are summarized and saved in Excel files for further analysis.

In [5]:

order_report = pd.read_excel("Company X - Order Report.xlsx")
pincode_mapping = pd.read_excel("Company X - Pincode Zones.xlsx")
sku_master = pd.read_excel("Company X - SKU Master.xlsx")

courier_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
charge_rate_card = pd.read_excel("Courier Company - Rates.xlsx")

order_report.rename(columns={"ExternOrderNo": "Order ID", "Order Qty": "Quantity"}, inplace=True)

merged_data = pd.merge(order_report, courier_invoice, on="Order ID", how="inner")

order_details = pd.merge(order_report, sku_master, on='SKU', how='left')
order_details['Total_Weight'] = order_details['Quantity'] * (order_details['Weight (g)'] / 1000)

total_weight_per_order = order_details.groupby('Order ID')['Total_Weight'].sum().reset_index()

merged_data = pd.merge(merged_data, total_weight_per_order, on="Order ID", how="left")

def calculate_weight_slab(row):
    if row['Total_Weight'] <= 0.5:
        return 0.5
    elif row['Total_Weight'] <= 1:
        return 1
    else:
        return 2

merged_data['Weight Slab'] = merged_data.apply(calculate_weight_slab, axis=1)

merged_data['Total Weight Match'] = merged_data['Total_Weight'] == merged_data['Charged Weight']
merged_data['Weight Slab Match'] = merged_data['Weight Slab'] == merged_data['Zone']

def calculate_expected_charges(total_weight, zone):
    rate_card = {'A': 10, 'B': 15, 'C': 20}
    rate = rate_card.get(zone, 20)
    expected_charges = rate * total_weight
    return expected_charges

merged_data['Expected Charges'] = merged_data.apply(lambda row: calculate_expected_charges(row['Total_Weight'], row['Zone']), axis=1)

comparison_results = merged_data[['Order ID', 'Total_Weight', 'Weight Slab', 'Charged Weight', 'Zone', 'Expected Charges', 'Billing Amount (Rs.)']]

comparison_results['Difference'] = comparison_results['Expected Charges'] - comparison_results['Billing Amount (Rs.)']

comparison_results.to_csv("Comparison_Results.csv", index=False)

overcharged_orders = (comparison_results['Difference'] < 0).sum()
correctly_charged_orders = (comparison_results['Difference'] == 0).sum()
undercharged_orders = (comparison_results['Difference'] > 0).sum()

total_overcharged_amount = comparison_results[comparison_results['Difference'] < 0]['Difference'].sum()
total_undercharged_amount = comparison_results[comparison_results['Difference'] > 0]['Difference'].sum()

summary_data = {
    'Count': [len(comparison_results), overcharged_orders, correctly_charged_orders, undercharged_orders],
    'Amount (Rs.)': [
        comparison_results['Billing Amount (Rs.)'].sum(),
        total_overcharged_amount,
        0,
        total_undercharged_amount
    ]
}
summary_index = ['Total Orders', 'Total Orders - Overcharged', 'Total Orders - Correctly Charged', 'Total Orders - Undercharged']
summary_table = pd.DataFrame(summary_data, index=summary_index)
# Final summary table
summary_table.to_excel("Summary_Table.xlsx")


# Getting Order_Level Calculations with Correct columns names according to question
merged_data['AWB Number'] = merged_data['AWB Code']
merged_data['Total weight as per X (KG)'] = merged_data['Total_Weight']
merged_data['Weight slab as per X (KG)'] = merged_data['Weight Slab']
merged_data['Total weight as per Courier Company (KG)'] = merged_data['Charged Weight']
merged_data['Weight slab charged by Courier Company (KG)'] = merged_data['Zone']
merged_data['Delivery Zone as per X'] = merged_data['Warehouse Pincode']
merged_data['Delivery Zone charged by Courier Company'] = merged_data['Customer Pincode']
merged_data['Expected Charge as per X (Rs.)'] = merged_data['Expected Charges']
merged_data['Charges Billed by Courier Company (Rs.)'] = merged_data['Billing Amount (Rs.)']
merged_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = merged_data['Expected Charges'] - merged_data['Billing Amount (Rs.)']

merged_data = merged_data[['Order ID', 'AWB Number', 'Total weight as per X (KG)', 'Weight slab as per X (KG)',
                           'Total weight as per Courier Company (KG)', 'Weight slab charged by Courier Company (KG)',
                           'Delivery Zone as per X', 'Delivery Zone charged by Courier Company',
                           'Expected Charge as per X (Rs.)', 'Charges Billed by Courier Company (Rs.)',
                           'Difference Between Expected Charges and Billed Charges (Rs.)']]
# Final Order Level Calculations
merged_data.to_excel("Order_Level_Calculations.xlsx", index=False)


# Data

In [8]:

print("order_report: ", order_report.head())
print("pincode_mapping: ", pincode_mapping.head())
print("sku_master: ", sku_master.head())

print("courier_invoice: ", courier_invoice.head())
print("charge_rate_card: ", charge_rate_card.head())

order_report:       Order ID            SKU  Quantity
0  2001827036  8904223818706       1.0
1  2001827036  8904223819093       1.0
2  2001827036  8904223819109       1.0
3  2001827036  8904223818430       1.0
4  2001827036  8904223819277       1.0
pincode_mapping:     Warehouse Pincode  Customer Pincode Zone
0             121003            507101    d
1             121003            486886    d
2             121003            532484    d
3             121003            143001    b
4             121003            515591    d
sku_master:               SKU  Weight (g)
0  8904223815682         210
1  8904223815859         165
2  8904223815866         113
3  8904223815873          65
4  8904223816214         120
courier_invoice:          AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003

# Final Results

In [11]:
merged_data.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,1091122418320,2.176,2.0,1.6,b,121003,173213,43.52,117.9,-74.38
1,2001827036,1091122418320,2.176,2.0,1.6,b,121003,173213,43.52,117.9,-74.38
2,2001827036,1091122418320,2.176,2.0,1.6,b,121003,173213,43.52,117.9,-74.38
3,2001827036,1091122418320,2.176,2.0,1.6,b,121003,173213,43.52,117.9,-74.38
4,2001827036,1091122418320,2.176,2.0,1.6,b,121003,173213,43.52,117.9,-74.38
